In [1]:
import tensorflow as tf
gpu_check = tf.test.gpu_device_name()
#print(tf.test.gpu_device_name(),'\n')
if 'GPU' in gpu_check:
    print("Yes, we have GPU!")
else:
    print("No GPU.")

Yes, we have GPU!


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
pwd

'/gdrive'

In [0]:
cd 'My Drive/data'

/gdrive/My Drive/data


In [0]:
!ls *txt

pathfinder_deadly_mine.txt	shakespeare_final.txt
pathfinder_token_sequences.txt	textgen_pathfinder.txt


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gensim
import string

import textwrap

from keras.layers import LSTM, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint

from keras.utils.data_utils import get_file
from keras.models import load_model
import pickle

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


Using TensorFlow backend.


In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# load
in_filename = 'data/pathfinder_token_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [5]:
sentences = [line.split() for line in lines]
#sentences[:2]

In [6]:
word_model = pickle.load(open('model/pathfinder_token_w2v300_word_model.pkl', 'rb'))
pretrained_weights = pickle.load(open('model/pathfinder_token_w2v300_weights.pkl', 'rb'))

tokenizer = pickle.load(open('model/pathfinder_token_tokenizer.pkl', 'rb'))

#model = load_model('model/pathfinder_token_w2v_model_100_epoch.h5')
#model = pickle.load(open('pathfinder_token_w2v_model_100_epoch.pkl', 'rb'))

#history = pickle.load(open('model/pathfinder_token_w2v_model_100_epoch_history.pkl', 'rb'))

#model.load_weights("model/pathfinder_token_w2v_model_100_epoch_weights.h5")

In [7]:
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

Result embedding shape: (2925, 300)


/home/ringoshin/.Envs/pfbot/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [8]:
sequences = tokenizer.texts_to_sequences(lines)

In [9]:
# define model
model = Sequential()
#model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))

model.add(LSTM(emdedding_size, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(emdedding_size))
model.add(Dropout(0.2))

model.add(Dense(emdedding_size, activation='relu'))

model.add(Dense((vocab_size+1), activation='softmax'))
print(model.summary())











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         877500    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 300)         721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300 

In [10]:
model.load_weights("model/pathfinder_token_w2v_model_100_epoch_weights.h5")

In [0]:
print('\nTraining word2vec...')
# workers=1 will ensure a fully deterministrically-reproducible run, per Gensim docs
word_model = gensim.models.Word2Vec(sentences, size=300, min_count=1, window=5, iter=100, workers=1)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)


Training word2vec...
Result embedding shape: (2925, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [11]:
print('Checking similar words:')
for word in ['gold', 'goblin', 'magic', 'priest', 'mine']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

Checking similar words:
  gold -> studded (0.31), empathize (0.26), not (0.25), communicate (0.25), here (0.24), investigating (0.23), provide (0.23), engraved (0.23)
  goblin -> warriors (0.54), reckless (0.47), tries (0.40), fights (0.36), pound (0.36), ac (0.36), acquired (0.34), accessible (0.33)
  magic -> largest (0.37), korvosan (0.36), minor (0.36), acadamae (0.34), radiate (0.33), torches (0.30), initially (0.30), identifies (0.29)
  priest -> sole (0.82), birm (0.80), curate (0.66), amelon (0.65), temple (0.54), desna (0.48), archways (0.45), sun (0.43)
  mine -> hillside (0.30), energy (0.29), movement (0.26), lowdown (0.25), second (0.23), cracked (0.22), managed (0.22), arts (0.22)


/home/ringoshin/.Envs/pfbot/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [13]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [14]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [15]:
def sample_word(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [16]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words, temperature=0):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        #   [0] coz shape is (num_lines, num_tokens_per_line)
        #   and there is only one line right now
        #   also num_tokens is not a const, hence trunc will be required
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed lenght
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        #yhat = model.predict_classes(encoded, verbose=0)
        yhat = sample_word(model.predict(encoded,verbose=0)[-1], temperature)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [18]:
# load cleaned text sequences
in_filename = 'data/pathfinder_token_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

In [0]:
# load the model
#model = load_model('pathfinder_token_model_300_epoch.h5')
#model = load_model('pathfinder_token_model_250_epoch.h5')
#model = load_model('pathfinder_token_model_150_epoch.h5')
#model = load_model('pathfinder_token_model_200_epoch.h5')
model = load_model('pathfinder_token_w2v_model_100_epoch.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
# load the tokenizer
tokenizer = pickle.load(open('model/pathfinder_token_tokenizer.pkl', 'rb'))

In [22]:
len(lines)
lines[:5]

['the giant black widow spider is a tough cr poisonous spider the marsh dwelling lizardfolk can be dangerous enemies or cautiously the flesh eating minotaur is a tough solo opponent for higher level characters or the deadly boss of a tribe of orcs or goblins the wererat is a sneaky shapeshifter',
 'giant black widow spider is a tough cr poisonous spider the marsh dwelling lizardfolk can be dangerous enemies or cautiously the flesh eating minotaur is a tough solo opponent for higher level characters or the deadly boss of a tribe of orcs or goblins the wererat is a sneaky shapeshifter relying',
 'black widow spider is a tough cr poisonous spider the marsh dwelling lizardfolk can be dangerous enemies or cautiously the flesh eating minotaur is a tough solo opponent for higher level characters or the deadly boss of a tribe of orcs or goblins the wererat is a sneaky shapeshifter relying on',
 'widow spider is a tough cr poisonous spider the marsh dwelling lizardfolk can be dangerous enemies 

In [23]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(textwrap.fill('%s' % (seed_text), 80) + '\n')

night air emanating from the graveyard west of town and because of their
superstition refuse to go out at night rumors suggest that another of the
twilight fell rituals must have gone awry stoic and resigned the peasants seem
to accept their self imposed curfew as their lot in life and



In [24]:
#seed_text='lycanthropes living in the area albeit fearfully and in hiding run a farmer off his or her land so they could move in So farthough a few savagely mauled farm animals have not been sufficient motivation to scare off any of the farmers and Shala has decided it time to try a more'

In [25]:
#seed_text='fearfully and in hiding run a farmer off his or her land so they could move in so farthough'

In [26]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 75, 1.5)
print(textwrap.fill('%s' % (generated), 80))

/home/ringoshin/.Envs/pfbot/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


give any signs that they feel anything needs to change if captain silva any pc
who elaborates on the battle if he full behind them the candle of night for the
relics and making an levitate voice gp to gold elements in the watch the envoy
at a cleric of potions from her desk the bottle contains a strong throat and a
document alcohol that on the false scepter upon a bottle labeled discovering
that


In [27]:
special_gen="""few weeks ago and have not yet returned the three missing monks are piotr oja
ollie lenstraand tibaldo russo brother joachim informs the pcs there is a cave
that has been used for shelter in the mountains along the route to galduria and
describes how the pcs can locate it as the pcs travel further north they can see
and feel approach small patches of snow linger in shaded areas the grasses are a
dull"""

In [28]:
generated = special_gen
print(textwrap.fill('%s' % (generated), 80))

few weeks ago and have not yet returned the three missing monks are piotr oja
ollie lenstraand tibaldo russo brother joachim informs the pcs there is a cave
that has been used for shelter in the mountains along the route to galduria and
describes how the pcs can locate it as the pcs travel further north they can see
and feel approach small patches of snow linger in shaded areas the grasses are a
dull


In [0]:
#!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 50.8MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=4a5fa3714e13de0fd9a3bc1bc9c2007859cb752ec07b8662f352dec45e0bb59f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1mz0tldt/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [29]:
import spacy
from spacy import displacy
from collections import Counter
#import en_core_web_sm
#nlp = en_core_web_sm.load()
import en_core_web_lg
nlp = en_core_web_lg.load()


In [30]:
sentence_nlp = nlp(generated)

# print named entities in article
print([(word, word.ent_type_) for word in sentence_nlp if word.ent_type_])

# visualize named entities
displacy.render(sentence_nlp, style='ent', jupyter=True)

[(few, 'DATE'), (weeks, 'DATE'), (ago, 'DATE'), (three, 'CARDINAL'), (piotr, 'PERSON'), (oja, 'PERSON'), (
, 'PERSON'), (ollie, 'PERSON'), (tibaldo, 'PERSON'), (russo, 'PERSON'), (joachim, 'PERSON'), (galduria, 'GPE')]


In [31]:
#[(x.orth_,x.pos_, x.lemma_) for x in [y 
#                                      for y
#                                      in nlp(generated)
#                                      if (not y.is_stop and y.pos_ != 'PUNCT') or y.pos_ == 'NUM']]
noun_list = []
for token in nlp(generated):
#    if (not token.is_stop and token.pos_ != 'PUNCT') or token.pos_ == 'NUM':
    if (token.pos_ == 'NOUN'):
        noun_list.append(token)
        print((token.orth_,token.pos_, token.lemma_))

('weeks', 'NOUN', 'week')
('monks', 'NOUN', 'monk')
('brother', 'NOUN', 'brother')
('pcs', 'NOUN', 'pc')
('cave', 'NOUN', 'cave')
('shelter', 'NOUN', 'shelter')
('mountains', 'NOUN', 'mountain')
('route', 'NOUN', 'route')
('pcs', 'NOUN', 'pc')
('pcs', 'NOUN', 'pc')
('approach', 'NOUN', 'approach')
('patches', 'NOUN', 'patch')
('snow', 'NOUN', 'snow')
('areas', 'NOUN', 'area')
('grasses', 'NOUN', 'grass')


In [32]:
#print([(i, noun) for i, noun in enumerate(noun_list)])
#print(doc[7], doc[8], doc[7].similarity(doc[8]))
test_doc = nlp('cave shelter mountains snow brother')
print(test_doc[0], test_doc[1], test_doc[0].similarity(test_doc[1]))
print(test_doc[0], test_doc[2], test_doc[0].similarity(test_doc[2]))
print(test_doc[1], test_doc[2], test_doc[1].similarity(test_doc[2]))
print(test_doc[2], test_doc[3], test_doc[2].similarity(test_doc[3]))
print(test_doc[0], test_doc[4], test_doc[0].similarity(test_doc[4]))

cave shelter 0.3689129
cave mountains 0.5231152
shelter mountains 0.3393006
mountains snow 0.5963584
cave brother 0.22205521


In [33]:
vocab_dict = word_model.wv.vocab

In [34]:
vocab = [k for k in vocab_dict.keys()]

In [35]:
len(vocab)

2925

In [0]:
import gensim.downloader as api
wv_glove = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
for word in ['gold', 'goblin', 'magic', 'priest', 'mine']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in wv_glove.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

In [0]:
for word in ['zimandi', 'goblin', 'magic', 'priest', 'mine']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

In [0]:
not_found_list, name_list, oov_list = [], [], []
skip_words = ['darkvision', 'bracers', 'dungeoneering', 'ungracefully', 'lycanthropic', 'starknife' ]
for word,i in word_model.wv.vocab.items():
    try:
        embedding_vector = wv_glove.wv[word] 
    except:
        not_found_list.append(word)
        if word not in skip_words:
            name_list.append(word)
        else:
            oov_list.append(word)
        #print(word, 'not found')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [0]:
name_list

In [36]:
doc = nlp(generated)
for chunk in doc.noun_chunks:
    print("{} | {} | {} | {}".format(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text))

the three missing monks | monks | nsubj | are
piotr oja | oja | attr | are
ollie lenstraand tibaldo russo brother | brother | nsubj | informs
joachim | joachim | appos | brother
the pcs | pcs | dobj | informs
a cave | cave | attr | is
that | that | nsubjpass | used
shelter | shelter | pobj | for
the mountains | mountains | pobj | in
the route | route | pobj | along
galduria | galduria | pobj | to
the pcs | pcs | nsubj | locate
it | it | dobj | locate
the pcs | pcs | nsubj | travel
they | they | nsubj | see
small patches | patches | nsubj | linger
snow | snow | pobj | of
shaded areas | areas | pobj | in
the grasses | grasses | nsubj | are


In [38]:
import spacy
from spacy.tokens import Span
from spacy.matcher import Matcher, PhraseMatcher

nlp = spacy.load("en_core_web_lg")
doc = nlp(u"FB is hiring a new Vice President of global policy")
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# the model didn't recognise "FB" as an entity :(

DATE = doc.vocab.strings[u"DATE"]  # get hash value of entity label
ORG = doc.vocab.strings[u"ORG"]  # get hash value of entity label
fb_ent = Span(doc, 0, 1, label='GOD') # create a Span for the new entity
doc.ents = list(doc.ents) + [fb_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('After', ents)
# [(u'FB', 0, 2, 'MOG')]

Before [('FB', 0, 2, 'ORG')]


ValueError: [E103] Trying to set conflicting doc.ents: '(0, 1, 'ORG')' and '(0, 1, 'GOD')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

In [39]:
god_list = ['Erastil', 'Aroden', 'Desna', 'Sarenrae']
race_list = ['Azlanti', 'Varisian', 'Thassilonian', 'Korvosan', 'Magnimarian']
org_list = ['Runelord', 'Runelords', 'Rockwhelp', 'Academae', 'Versade', 
            'Pathfinder', 'Society']
monster_list = ['wererat', 'wererats', 'werewolf', 'Yarg', 'goblin', 'goblins', 
                'orc', 'orcs', 'spider', 'spiders', 'minotaur', 'elemental', 
                'wolf', 'lycanthropes', 'lycanthrope', 'Cheh', 'Kavoos', 'Shala',
                'undead', 'ghouls']
person_list = ['Eberius', 'Tauranor', 'Xanderghul', 'Sheila', 'Heidmarch',
               'Hork', 'Shalesmash', 'Amelon', 'Birm', 'Grald', 'Hazelindra', 
               'Linna', 'Montrovale', 'Gorbic', 'Salmore', 'Mena', 'Zamola', 'Duso', 'Galino',
               'Piotr', 'Oja', 'Ollie', 'Lenstraand', 'Tibaldo', 'Russo', 'Joachim',
               'Aldemar', 'Lenstra', 'Jargie', 'Krzysztof', 'Szabo', 'Shabana', 'Neergah', 
               'Elliana', 'Silva', 'Savasha', 'Versade', 'Nindrik', 'Versade', 'Hobart', 'Deverin',
               'Kendra', 'Deverin', 'Gradon', 'Scarnetti', 'Zimandi', 'Kaddren', 'Cheiskaia', 'Nirodin',
               'Gradon', 'Scarnetti', 'Tauk', 'Yordan', 'Zorakov', 'Lucas', 'Gustavo', 'Kantaro',
               'Das', 'Korvut', 'Zanthus', 'Belor', 'Durn', 'Belor', 'Hemlock', 'Pavlina']
location_list = ['Golarion', 'Varisia', 'Lurkwood', 'Riddleport', 'Galduria', 
                 'Acadamae', 'Korvosa', 'Thassilon',  'Magnimar', 'Irespan', 
                 'Dockway', 'Sandpoint', 'Brinestump', 'Marsh', 'Soggy', 'River', 
                 'Windsong', 'Abbey', 'Rusty', 'Dragon', 'Necropolis']
spell_list = ['Burning', 'Hands']
long_name_list = god_list + race_list + org_list + monster_list + person_list + location_list + spell_list


In [40]:
god_labels = ['Erastil', 'Aroden', 'Desna', 'Sarenrae']
race_labels = ['Azlanti', 'Varisian', 'Thassilonian', 'Korvosan', 'Magnimarian']
org_labels = ['Runelord', 'Runelords', 'Aockwhelp', 'Academae', 'Versade',
             'Pathfinder Society']
monster_labels = ['wererat', 'wererats', 'werewolf', 'Yarg', 'goblin', 'goblins', 
                'orc', 'orcs', 'spider', 'spiders', 'minotaur', 'elemental', 
                'wolf', 'lycanthropes', 'lycanthrope', 'Cheh', 'Kavoos', 'Shala',
                'undead', 'ghouls']
person_labels = ['Eberius Tauranor', 'Xanderghul', 'Sheila Heidmarch',
#               'Hork Shalesmash', 'Amelon Birm', 'Grald', 'Hazelindra', 
               'Hork', 'Amelon Birm', 'Grald', 'Hazelindra', 
               'Linna Montrovale', 'Gorbic Salmore', 'Mena Zamola', 'Duso Galino',
               'Piotr Oja', 'Ollie Lenstraand', 'Tibaldo Russo', 'Joachim',
               'Aldemar Lenstra', 'Jargie', 'Krzysztof Szabo', 'Shabana Neergah', 
               'Elliana Silva', 'Savasha Versade', 'Nindrik Versade', 'Hobart Deverin',
               'Kendra Deverin', 'Gradon Scarnetti', 'Zimandi Kaddren', 'Cheiskaia Nirodin',
               'Gradon Scarnetti', 'Tauk', 'Yordan Zorakov', 'Lucas', 'Gustavo', 'Kantaro',
               'Das Korvut', 'Zanthus', 'Belor', 'Durn', 'Belor Hemlock', 'Pavlina']
location_labels = ['Golarion', 'Varisia', 'Lurkwood', 'Riddleport', 'Galduria', 
                 'Acadamae', 'Korvosa', 'Thassilon',  'Magnimar', 'Irespan', 
                 'Dockway', 'Sandpoint', 'Brinestump Marsh', 'Soggy River', 'Windsong Abbey', 
                  'Rusty Dragon', 'Necropolis']
spell_labels = ['Burning Hands']

In [41]:
def Entity_Type(word):
    word_lower = word
    word = word[0].upper() + word[1:]
    if word in god_list:
        return 'GOD', word
    elif word in race_list:
        return 'RACE', word
    elif word in org_list:
        return 'ORG', word
    elif word in monster_list:
        return 'MOB', word
    elif word_lower in monster_list:
        return 'MOB', word_lower 
    elif word in person_list:
        return 'PER', word
    elif word in location_list:
        return 'LOC', word
    elif word in spell_list:
        return 'SP', word
    else:
        return 'UNK', word_lower

In [42]:
test_words = ['sarenrae', 'was', 'mentioned', 'but', 'tauranor', 'did', 'not', 'think', 'goblins', 'are','in', 'korvosa']
for word in test_words:
    print("{}: {}".format(word, Entity_Type(word)))

sarenrae: ('GOD', 'Sarenrae')
was: ('UNK', 'was')
mentioned: ('UNK', 'mentioned')
but: ('UNK', 'but')
tauranor: ('PER', 'Tauranor')
did: ('UNK', 'did')
not: ('UNK', 'not')
think: ('UNK', 'think')
goblins: ('MOB', 'goblins')
are: ('UNK', 'are')
in: ('UNK', 'in')
korvosa: ('LOC', 'Korvosa')


In [43]:
generated = 'so sarenrae is mentioned adn there are goblins too in Korvosa praying to runelords and varisian and kantaro ' + generated

In [44]:
generated='of aroden before he rose to godhood thousands of years ago while the veracity of the origin remains unconfirmed the arcane spells within are worth investigation by the pathfinder society and perhaps the diviners in the organization can determine the true nature to make good her escape she contacted venture'

In [45]:
g_new = ''
g_god = []
g_mob = []
g_per = []
g_loc = []
g_race = []
g_org = []
g_sp = []
for g_word in generated.split():
    g_type, g_word = Entity_Type(g_word)
    if g_type=='GOD':
        g_god.append(g_word)
    elif g_type=='MOB':
        g_mob.append(g_word)
    elif g_type=='PER':
        g_per.append(g_word)
    elif g_type=='LOC':
        g_loc.append(g_word)
    elif g_type=='RACE':
        g_race.append(g_word)
    elif g_type=='ORG':
        g_org.append(g_word)
    elif g_type=='SP':
        g_sp.append(g_word)
    g_new  = ' '.join([g_new, g_word]) 
g_new = g_new.strip()

In [46]:
g_new

'of Aroden before he rose to godhood thousands of years ago while the veracity of the origin remains unconfirmed the arcane spells within are worth investigation by the Pathfinder Society and perhaps the diviners in the organization can determine the true nature to make good her escape she contacted venture'

In [47]:
new_g_per=[]
if len(g_per)>1:
    x = iter(g_per)
    first_name = next(x)
    prev_fullname = ''
    for _ in range(len(g_per)-1):
        second_name = next(x)
        full_name = '{} {}'.format(first_name, second_name)
        if full_name in person_labels:
            new_g_per.append(full_name)
            prev_fullname = full_name
        else:
            if first_name in prev_fullname:
                prev_fullname = ''
            else:
                new_g_per.append(first_name)
        first_name = second_name
    g_per = new_g_per
    if first_name not in prev_fullname:
        g_per.append(first_name)

In [48]:
g_per

[]

In [49]:
new_g_loc=[]
if len(g_loc)>1:
    x = iter(g_loc)
    first_name = next(x)
    prev_fullname = ''
    for _ in range(len(g_loc)-1):
        second_name = next(x)
        full_name = '{} {}'.format(first_name, second_name)
        if full_name in location_labels:
            new_g_loc.append(full_name)
            prev_fullname = full_name
        else:
            if first_name in prev_fullname:
                prev_fullname = ''
            else:
                new_g_loc.append(first_name)
        first_name = second_name
    g_loc = new_g_loc
    if first_name not in prev_fullname:
        g_loc.append(first_name)

In [50]:
g_loc

[]

In [51]:
new_g_sp=[]
if len(g_sp)>1:
    x = iter(g_sp)
    first_name = next(x)
    prev_fullname = ''
    for _ in range(len(g_sp)-1):
        second_name = next(x)
        full_name = '{} {}'.format(first_name, second_name)
        if full_name in spell_labels:
            new_g_sp.append(full_name)
            prev_fullname = full_name
        else:
            if first_name in prev_fullname:
                prev_fullname = ''
            else:
                new_g_sp.append(first_name)
        first_name = second_name
    g_sp = new_g_sp
    if first_name not in prev_fullname:
        g_sp.append(first_name)

In [52]:
g_sp

[]

In [53]:
new_g_org=[]
if len(g_org)>1:
    x = iter(g_org)
    first_name = next(x)
    prev_fullname = ''
    for _ in range(len(g_org)-1):
        second_name = next(x)
        full_name = '{} {}'.format(first_name, second_name)
        if full_name in org_labels:
            new_g_org.append(full_name)
            prev_fullname = full_name
        else:
            if first_name in prev_fullname:
                prev_fullname = ''
            else:
                new_g_org.append(first_name)
        first_name = second_name
    g_org = new_g_org
    if first_name not in prev_fullname:
        g_org.append(first_name)

In [54]:
g_org

['Pathfinder Society']

In [55]:
def List_Notables():
    g_all = g_god + g_mob + g_per + g_loc + g_race + g_org
    if g_all:
        print('Notable(s) found:')
        if g_god:
            print("   God(s)     : {}".format(', '.join(g_god)))
        if g_mob:
            print("   Monster(s) : {}".format(', '.join(g_mob)))
        if g_per:
            print("   Person(s)  : {}".format(', '.join(g_per)))
        if g_sp:
            print("   Spell(s)   : {}".format(', '.join(g_sp)))
        if g_loc:
            print("   Location(s): {}".format(', '.join(g_loc)))
        if g_race:
            print("   Race(s)    : {}".format(', '.join(g_race)))
        if g_org:
            print("   Organisation(s): {}".format(', '.join(g_org)))
    else:
        print(">>> No notables found.")

In [56]:
List_Notables()

Notable(s) found:
   God(s)     : Aroden
   Organisation(s): Pathfinder Society


In [57]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [58]:
god_patterns = [nlp(text) for text in god_labels]
mob_patterns = [nlp(text) for text in monster_labels]
per_patterns = list(nlp.pipe(person_labels))
loc_patterns = list(nlp.pipe(location_labels))
race_patterns = [nlp(text) for text in race_labels]
org_patterns = [nlp(text) for text in org_labels]
sp_patterns = [nlp(text) for text in spell_labels]

matcher = PhraseMatcher(nlp.vocab)
matcher.add('GOD', None, *god_patterns)
matcher.add('MOB', None, *mob_patterns)
matcher.add('PER', None, *per_patterns)
matcher.add('LOC', None, *loc_patterns)
matcher.add('RACE', None, *race_patterns)
matcher.add('ORG', None, *org_patterns)
matcher.add('SP', None, *sp_patterns)

In [59]:
doc = nlp(g_new)
matches = matcher(doc)
spans = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    print(rule_id, span.text)
    spans.append(Span(doc, start, end, label=rule_id))
    doc.ents = spans

GOD Aroden
ORG Pathfinder Society


In [60]:
print()
print()
print('You: {}'.format('Tell me about a tome penned by an apprentice.'))
print('-'*95)
options = {"ents": ['GOD','MOB','PER','LOC','RACE','ORG','SP'],
           "colors": {'GOD':'#f2865e','MOB':'#58f549','PER':'#aef5ef',
                      'LOC':'pink','RACE':'#edcb45','ORG':'#d88fff', 'SP':'pink'}}
print('Snaug_bot:') 
displacy.render(doc, style='ent', jupyter=True, options=options)
print()
List_Notables()
print('-'*95)
print()
print()



You: Tell me about a tome penned by an apprentice.
-----------------------------------------------------------------------------------------------
Snaug_bot:



Notable(s) found:
   God(s)     : Aroden
   Organisation(s): Pathfinder Society
-----------------------------------------------------------------------------------------------


